# Make the Gregory style plot for C1 scenarios for TCRE and ZEC from CO2 only runs

In [ ]:
import logging
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
from scipy.stats import linregress

In [ ]:
os.makedirs('../plots', exist_ok=True)

In [ ]:
logger = logging.getLogger('fair')
logger.setLevel(level=logging.CRITICAL)

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
emis_future = np.load("../data/co2_emissions_c1.npy")  # C1 emissions scenarios starting in 1995

In [ ]:
emis_future.shape

In [ ]:
# First, remove three scenarios which don't reach net zero
emis_future[-1,:] > 0

In [ ]:
emis_future = emis_future[:,emis_future[-1,:] < 0]

In [ ]:
emis_future.shape

In [ ]:
# then, ensure we pull in the full historical
emis = np.ones((351, 94)) * np.nan

In [ ]:
df_hist = pd.read_csv('../data/historical_emissions.csv')
emis_hist = (
    df_hist.loc[df_hist['Variable']=='Emissions|CO2|MAGICC AFOLU', '1750':'1994'].values +
    df_hist.loc[df_hist['Variable']=='Emissions|CO2|MAGICC Fossil and Industrial', '1750':'1994'].values
).squeeze()/1000

In [ ]:
emis[:245, :] = emis_hist[:, None]

In [ ]:
emis[245:, :] = emis_future

In [ ]:
first_negative_year = np.argmin(emis>0, axis=0)

In [ ]:
first_negative_year  # bear in mind these are "timepoints" so are year+.5

In [ ]:
# the appropriate timebound is the one with the same label as first_negative_year, occurring at the whole number year
emis[first_negative_year[0]-1, 0], emis[first_negative_year[0], 0]

In [ ]:
# cumulative net negative CO2 emissions from year of net zero to 2100 (GtCO2)
cumulative_positive_1750_to_nz = np.ones(94) * np.nan
cumulative_negative_nz_to_2100 = np.ones(94) * np.nan

for i, year in enumerate(first_negative_year):
    cumulative_positive_1750_to_nz[i] = np.cumsum(emis[:year, i])[-1] / 1000
    cumulative_negative_nz_to_2100[i] = np.cumsum(emis[year:, i])[-1] / 1000

In [ ]:
# this suggests that the split is correct
np.cumsum(emis[:year, i])[-1]/1000, np.cumsum(emis[year:, i])[0] / 1000

In [ ]:
cumulative_positive_1750_to_nz  # TtCO2 each scenario (x94) 1750 to net zero

In [ ]:
cumulative_negative_nz_to_2100  # unit: TtCO2 in each scenario (x94) from net zero to 2100

## now run fair and calculate warming at different points
- 1750 to peak T (positive phase)
- 1750 to T @ net zero (positive phase)
- peak T to end of century temperature (negative phase)
- T @ net zero to end of century (negative phase)

As with cumulative emissions, the net zero point should be determined using the same label as net zero year (timebound before timepoint where first negative value encountered)

In [ ]:
f = FAIR()
scenarios = np.arange(94)
f.define_scenarios(scenarios)

In [ ]:
fair_params_1_4_1_file = '../data/calibrated_constrained_parameters_1.4.1.csv'
fair_species_configs_1_4_1_file = '../data/species_configs_properties_1.4.1.csv'

In [ ]:
df_configs = pd.read_csv(fair_params_1_4_1_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)

In [ ]:
species = ['CO2', 'CH4', 'N2O']
properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
f.define_species(species, properties)

In [ ]:
f.define_time(1750, 2101, 1)

In [ ]:
f.allocate()

In [ ]:
f.concentration.loc[dict(specie="CH4")] = 729.2
f.concentration.loc[dict(specie="N2O")] = 270.1

In [ ]:
f.emissions.loc[dict(specie="CO2")] = emis[..., None]

In [ ]:
f.fill_species_configs(fair_species_configs_1_4_1_file)

In [ ]:
f.override_defaults(fair_params_1_4_1_file)

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

In [ ]:
f.temperature.sel(layer=0).shape

In [ ]:
temp_peakT_to_2100 = (
    f.temperature.sel(layer=0, timebounds=2101) -
    f.temperature.sel(layer=0).max(dim='timebounds')
).values  # convention negative for cooling

In [ ]:
#pl.plot(f.temperature.sel(layer=0, scenario=0));

In [ ]:
temp_1750_to_peakT = (
    f.temperature.sel(layer=0).max(dim='timebounds') -
    f.temperature.sel(layer=0, timebounds=1750)  # 1750 T should be zero
).values

In [ ]:
temp_peakT_to_2100

In [ ]:
temp_1750_to_peakT

In [ ]:
temp_nz_to_2100 = np.ones((94, 841)) * np.nan
for i in range(94):
    temp_nz_to_2100[i, :] = (
        f.temperature.sel(layer=0, scenario=i, timebounds=2101) -
        f.temperature.sel(layer=0, scenario=i, timebounds=1750+first_negative_year[i])
    ).values

temp_1750_to_nz = np.ones((94, 841)) * np.nan
for i in range(94):
    temp_1750_to_nz[i, :] = (
        f.temperature.sel(layer=0, scenario=i, timebounds=1750+first_negative_year[i]) - 
        f.temperature.sel(layer=0, scenario=i, timebounds=1750)
    ).values

In [ ]:
temp_nz_to_2100

In [ ]:
temp_1750_to_nz

In [ ]:
# TCRE_down is K per EgCO2: temperature divide cumulative emissions
# notice that this method doesn't work where ZEC is strongly positive, since 2101 is highest temperature, and values bunch at zero
pl.hist(temp_peakT_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=20)
pl.title('post-peak T negative TCRE (°C/TtCO2 removed, 1st scen.)')
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
pl.hist(temp_nz_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=20)
pl.title('post-netzero negative TCRE (°C/TtCO2 removed, 1st scen.)')
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
# we see the post net zero definition doesn't suffer from problem of positive ZEC
# in fact, ZEC is aliased into the definition, which is kind of what we want
pl.hist(temp_peakT_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=np.arange(-0.75, 1.3, 0.05), alpha=0.4, label='post peakT')
pl.hist(temp_nz_to_2100[0, :] / cumulative_negative_nz_to_2100[0], bins=np.arange(-0.75, 1.3, 0.05), alpha=0.4, label='post net zero')
pl.title('comparison defs. -ve TCRE (°C/TtCO2 removed, 1st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
pl.hist(temp_1750_to_peakT[0, :] / cumulative_positive_1750_to_nz[0], bins=np.arange(0.20, 0.82, 0.02), alpha=0.4, label='up to peakT')
pl.hist(temp_1750_to_nz[0, :] / cumulative_positive_1750_to_nz[0], bins=np.arange(0.20, 0.82, 0.02), alpha=0.4, label='up to net zero')
pl.title('comparison defs. +ve TCRE (°C/TtCO2 removed, 1st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
pl.hist(
    (temp_peakT_to_2100[0, :] / cumulative_negative_nz_to_2100[0]) / (temp_1750_to_peakT[0, :] / cumulative_positive_1750_to_nz[0]), 
bins=np.arange(-1.1, 2.7, 0.1), alpha=0.4, label='post peakT')
pl.hist(
    (temp_nz_to_2100[0, :] / cumulative_negative_nz_to_2100[0]) / (temp_1750_to_nz[0, :] / cumulative_positive_1750_to_nz[0]), 
bins=np.arange(-1.1, 2.7, 0.1), alpha=0.4, label='post netzero')
#pl.hist(temp_1750_to_nz[0, :] / cumulative_positive_1750_to_nz[0], bins=np.arange(-0.75, 1.3, 0.05), alpha=0.4, label='post net zero')
pl.title('comparison of negative/positive TCRE ratios (1st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
# we see where we hit net zero late, we have much larger uncertainty in TCRE since cumulative CO2 divisor is small
pl.hist(temp_peakT_to_2100[30, :] / cumulative_negative_nz_to_2100[30], bins=np.arange(-4, 5.1, 0.1), alpha=0.4, label='post peakT')
pl.hist(temp_nz_to_2100[30, :] / cumulative_negative_nz_to_2100[30], bins=np.arange(-4, 5.1, 0.1), alpha=0.4, label='post net zero')
pl.title('comparison defs. -ve TCRE (°C/TtCO2 removed, 31st scen.)')
pl.legend()
pl.xlabel('°C / (1000GtCO2)')

In [ ]:
# one TCRE estimate for every scenario/config; but scenarios look similar
tcre_up_at_net_zero = temp_1750_to_nz/cumulative_positive_1750_to_nz[:, None]

In [ ]:
tcre_up_at_net_zero.shape

In [ ]:
for i in range(94):
    pl.hist(tcre_up_at_net_zero[:, i], alpha=0.3)

In [ ]:
cmap = mpl.colormaps['plasma']
# Take colors at regular intervals spanning the colormap.
colors = cmap(np.linspace(0, 1, 841))

In [ ]:
def mapping_function(val, vmin, vmax):
    return (val - vmin) / (vmax - vmin)

In [ ]:
tcre_up_at_net_zero_scen_means = np.mean(tcre_up_at_net_zero, axis=0)
colors = cmap(mapping_function(tcre_up_at_net_zero_scen_means, tcre_up_at_net_zero_scen_means.min(), tcre_up_at_net_zero_scen_means.max()))

In [ ]:
# color by tcre_up_at_netzero
for i in range(841): # size/10
    pl.scatter(cumulative_negative_nz_to_2100, temp_peakT_to_2100[:, i], s=4, color=colors[i])
pl.title('All scenarios, each fair config');
pl.xlabel('netzero cum. removal (net zero to 2100, 1000GtCO2)');
pl.ylabel('warming, peak temperature to 2100');

In [ ]:
colors.shape

In [ ]:
(colors[:, None, :] * np.ones((1, 94, 1))).reshape(841*94, 4).shape

In [ ]:
temp_peakT_to_2100.shape

In [ ]:
# color by tcre_up_at_netzero
fig, ax = pl.subplots(figsize=((11/2.54, 9/2.54)))
sc = ax.scatter(
    (cumulative_negative_nz_to_2100[:, None] * np.ones((1, 841))).reshape(841*94).T, 
    temp_peakT_to_2100.reshape(841*94).T, 
    c=(
        (
            mapping_function(
                tcre_up_at_net_zero_scen_means, 
                tcre_up_at_net_zero_scen_means.min(), 
                tcre_up_at_net_zero_scen_means.max()
            ) * np.ones((94, 1))
        ).reshape(841*94).T
    ),
    s=4,
    cmap=cmap,
    vmin=tcre_up_at_net_zero_scen_means.min(),
    vmax=tcre_up_at_net_zero_scen_means.max()
)
fig.colorbar(sc, ax=ax, label="TCRE_up (K (TtCO$_2$)$^{-1}$)")
ax.set_title('All scenarios, each fair config');
ax.set_xlabel('netzero cum. removal (net zero to 2100, TtCO$_2$)');
ax.set_ylabel('warming, peak temperature to 2100');

In [ ]:
# each coloured band is a parameter set from fair (dim 841), each vertical line is a C1 scenario (dim 94)

# first calculate regression slope and intercept for each fair config set
sl_down_nz = np.zeros(841)
ic_down_nz = np.zeros(841)
r_down_nz = np.zeros(841)
for i in range(841):
    lr = linregress(cumulative_negative_nz_to_2100, temp_nz_to_2100[:, i])
    sl_down_nz[i], ic_down_nz[i], r_down_nz[i] = lr.slope, lr.intercept, lr.rvalue

# then plot the scenarios as before
fig, ax = pl.subplots(figsize=((11/2.54, 9/2.54)))
sc = ax.scatter(
    (cumulative_negative_nz_to_2100[:, None] * np.ones((1, 841))).reshape(841*94).T, 
    temp_nz_to_2100.reshape(841*94).T, 
    c=(tcre_up_at_net_zero_scen_means * np.ones((94, 1))).reshape(841*94).T,
    s=4,
    cmap=cmap,
    vmin=tcre_up_at_net_zero_scen_means.min(),
    vmax=tcre_up_at_net_zero_scen_means.max()
)
ax.axvline(0, color='k', ls=':', lw=0.5)
fig.colorbar(sc, ax=ax, label=r"TCRE$_{\mathrm{up}}$ (K (TtCO$_2$)$^{-1}$)")
ax.set_title('Cumulative CO$_2$ emissions v warming after net zero');
ax.set_xlabel('cumulative removal (net zero to 2100, TtCO$_2$)');
ax.set_ylabel('temperature, year of net zero to 2100 (K)');
fig.tight_layout()
pl.savefig('../plots/cooling_after_net_zero.pdf');
pl.savefig('../plots/cooling_after_net_zero.png');

In [ ]:
pl.hist(r_down_nz, bins=np.arange(0.85, 1.00, 0.01))

In [ ]:
# now plot the regression lines
fig, ax = pl.subplots(figsize=((11/2.54, 9/2.54)))
for i in range(841):
    ax.plot((-0.74, 0), (-0.74*sl_down_nz[i]+ic_down_nz[i], ic_down_nz[i]), color=colors[i])

fig.colorbar(sc, ax=ax, label="TCRE_up (K (TtCO$_2$)$^{-1}$)")
ax.set_title('All scenarios, each fair config');
ax.set_xlabel('netzero cum. removal (net zero to 2100, TtCO$_2$)');
ax.set_ylabel('warming, year of net zero to 2100 (K)');

In [ ]:
# plot TCREup v TCREdown
fig, ax = pl.subplots(figsize=((11/2.54, 9/2.54)))
sc = ax.scatter(
    tcre_up_at_net_zero_scen_means, 
    sl_down_nz, 
    c=tcre_up_at_net_zero_scen_means,
    s=4,
    cmap=cmap,
    vmin=tcre_up_at_net_zero_scen_means.min(),
    vmax=tcre_up_at_net_zero_scen_means.max()
)

# fig.colorbar(sc, ax=ax, label="TCRE_up (K (TtCO$_2$)$^{-1}$)")
ax.plot((0, 0.8), (0, 0.6), ls=':', color='k')
ax.plot((0, 0.8), (0, 0.8), ls='--', color='k')  # below, removals less efficient, above, more efficient
ax.plot((0, 0.8), (0, 1), ls=':', color='k')
ax.text(0.69, 0.69, '1:1', rotation=45, size=8, backgroundcolor="w", ha='center', va='center')
ax.text(0.55, 0.55*1.25, '1.25:1', rotation=np.degrees(np.arctan(5/4)), size=8, backgroundcolor="w", ha='center', va='center')
ax.text(0.7, 0.7*0.75, '0.75:1', rotation=np.degrees(np.arctan(3/4)), size=8, backgroundcolor="w", ha='center', va='center')
ax.set_title(r'TCRE$_{\mathrm{up}}$ v TCRE$_{\mathrm{down}}$');
ax.set_xlabel(r'TCRE$_{\mathrm{up}}$ (K (TtCO$_2$)$^{-1}$)');
ax.set_ylabel(r'TCRE$_{\mathrm{down}}$ (K (TtCO$_2$)$^{-1}$)');
ax.set_xlim(0.2, 0.75)
ax.set_ylim(0.2, 0.75)
fig.colorbar(sc, ax=ax, label=r"TCRE$_{\mathrm{up}}$ (K (TtCO$_2$)$^{-1}$)")
ax.text(0.22, 0.73, 'Removals more effective\nthan emissions', ha='left', va='top', color='b')
ax.text(0.73, 0.22, 'Removals less effective\nthan emissions', ha='right', va='bottom', color='r')
ax.fill_between(np.arange(0.2, 0.8, 0.05), 0, np.arange(0.2, 0.8, 0.05), color='r', alpha=0.1)
ax.fill_between(np.arange(0.2, 0.8, 0.05), np.arange(0.2, 0.8, 0.05), 1, color='b', alpha=0.1)

fig.tight_layout()
pl.savefig('../plots/tcre_up_tcre_down.pdf')
pl.savefig('../plots/tcre_up_tcre_down.png')

In [ ]:
np.degrees(np.arctan(5/4))

In [ ]:
# plot ZEC v TCREdown
fig, ax = pl.subplots(figsize=((11/2.54, 9/2.54)))
sc = ax.scatter(
    ic_down_nz, 
    sl_down_nz, 
    c=tcre_up_at_net_zero_scen_means,
    s=4,
    cmap=cmap,
    vmin=tcre_up_at_net_zero_scen_means.min(),
    vmax=tcre_up_at_net_zero_scen_means.max()
)

fig.colorbar(sc, ax=ax, label="TCRE_up (K (TtCO$_2$)$^{-1}$)")
ax.set_title('All scenarios, each fair config');
ax.set_xlabel('ZEC (K)');
ax.set_ylabel('TCRE_down (K (TtCO$_2$)$^{-1}$)');
ax.set_xlim(-0.2, 0.4)
ax.set_ylim(0.2, 0.75)

In [ ]:
# plot ZEC v TCREup
fig, ax = pl.subplots(figsize=((9/2.54, 9/2.54)))
sc = ax.scatter(
    ic_down_nz, 
    tcre_up_at_net_zero_scen_means, 
    c=tcre_up_at_net_zero_scen_means,
    s=4,
    cmap=cmap,
    vmin=tcre_up_at_net_zero_scen_means.min(),
    vmax=tcre_up_at_net_zero_scen_means.max()
)

# fig.colorbar(sc, ax=ax, label="TCRE_up (K (TtCO$_2$)$^{-1}$)")
ax.set_title('All scenarios, each fair config');
ax.set_xlabel('ZEC (K)');
ax.set_ylabel('TCRE_up (K (TtCO$_2$)$^{-1}$)');
ax.set_xlim(-0.2, 0.4)
ax.set_ylim(0.2, 0.75)

In [ ]:
# plot ZEC v TCREup
fig, ax = pl.subplots(figsize=((9/2.54, 9/2.54)))
sc = ax.scatter(
    sl_down_nz/tcre_up_at_net_zero_scen_means, 
    ic_down_nz, 
    c=tcre_up_at_net_zero_scen_means,
    s=4,
    cmap=cmap,
    vmin=tcre_up_at_net_zero_scen_means.min(),
    vmax=tcre_up_at_net_zero_scen_means.max()
)

# fig.colorbar(sc, ax=ax, label="TCRE_up (K (TtCO$_2$)$^{-1}$)")
ax.set_title('All scenarios, each fair config');
ax.set_ylabel('ZEC (K)');
ax.set_xlabel('TCRE_down/TCRE_up');
ax.set_ylim(-0.2, 0.4)
ax.set_xlim(0.5, 1.5)

In [ ]:
# Demonstrate that the way up is also linear (though the regression intercepts are not centred around zero)
sl_up_nz = np.zeros(841)
ic_up_nz = np.zeros(841)
for i in range(841):
    lr = linregress(cumulative_positive_1750_to_nz, temp_1750_to_nz[:, i])
    sl_up_nz[i], ic_up_nz[i] = lr.slope, lr.intercept

# then plot the scenarios as before
fig, ax = pl.subplots(figsize=((11/2.54, 9/2.54)))
sc = ax.scatter(
    (cumulative_positive_1750_to_nz[:, None] * np.ones((1, 841))).reshape(841*94).T, 
    temp_1750_to_nz.reshape(841*94).T, 
    c=(tcre_up_at_net_zero_scen_means * np.ones((94, 1))).reshape(841*94).T,
    s=4,
    cmap=cmap,
    vmin=tcre_up_at_net_zero_scen_means.min(),
    vmax=tcre_up_at_net_zero_scen_means.max()
)

fig.colorbar(sc, ax=ax, label=r"TCRE$_{\mathrm{up}}$ (K (TtCO$_2$)$^{-1}$)")
ax.set_title('Cumulative CO$_2$ emissions v warming at net zero');
ax.set_xlabel('cumulative emissions, 1750 to net zero (TtCO$_2$)');
ax.set_ylabel('temperature, 1750 to year of net zero (K)');

fig.tight_layout()
pl.savefig('../plots/warming_at_net_zero.png')
pl.savefig('../plots/warming_at_net_zero.pdf')

In [ ]:
# We're not using the regression estimate of TCREup because it's a little low and there's a substantial intercept.
# Just temperature at net zero divide cumulative emissions at net zero
# this plot is for comparison/diagnosis
pl.hist(sl_up_nz, bins=np.arange(0.1, 0.82, 0.02), alpha=0.4, label='TCREup')
pl.hist(sl_down_nz, bins=np.arange(0.2, 0.72, 0.02), alpha=0.4, label='TCRE_down')
pl.title('Regression based estimates of TCREup and TCREdown')
print(np.percentile(sl_up_nz, (5, 50, 95)))
print(np.percentile(sl_down_nz, (5, 50, 95)))
pl.legend()

In [ ]:
# here the regression based intercepts
pl.hist(ic_up_nz, bins=np.arange(-0.3, 1.02, 0.02), alpha=0.4, label='TCREup')
pl.hist(ic_down_nz, bins=np.arange(-0.3, 1.02, 0.02), alpha=0.4, label='TCRE_down')
pl.title('Regression-based ZEC estimates from netzero')
print(np.percentile(ic_up_nz, (5, 50, 95)))
print(np.percentile(ic_down_nz, (5, 50, 95)))
pl.legend()

In [ ]:
# suggestion is to calculate the positive TCRE either taking temperature at net zero divide cumulative emissions at net zero
# using 1pctCO2 runs is messy because ZEC itself is defined as a point estimate in 1pctCO2 runs
# I think the most approprite value to use for TCREup is the total warming at net zero divide cumulative emissions at net zero.
# then rely on the fact that ZEC in fair is ~zero, and take ratio of TCREdown and TCREup.
# also, this keeps some kind of internal scenario consistency.

In [ ]:
for iconf in range(11,13):
    pl.plot(np.cumsum(emis[:, 22]), 
        f.temperature.sel(
            timebounds=np.arange(1751, 2102), 
            layer=0, 
            scenario=22, 
            config=configs[iconf]
        ),
        # ) - f.temperature.sel(
        #     timebounds=2010,
        #     layer=0, 
        #     scenario=22, 
        #     config=configs[iconf]
        # ),
        color=colors[iconf]
    );

In [ ]:
cumulative_positive_1750_to_nz

In [ ]:
pl.hist(tcre_up_at_net_zero_scen_means)

In [ ]:
# throw in C / CO2 conversion factor here to compare with flat10 and values reported in IPCC and fair-calibrate papers
pl.hist(tcre_up_at_net_zero_scen_means * 3.664, bins=np.arange(0.3, 3.05, 0.05), alpha=0.4, label='TCRE_up')
pl.hist(sl_down_nz * 3.664, bins=np.arange(0.3, 3.05, 0.05), alpha=0.4, label='TCRE_down')
pl.title('TCRE_up at net zero and TCRE_down from regression')
print(np.percentile(tcre_up_at_net_zero_scen_means * 3.664, (5, 50, 95)))
print(np.percentile(sl_down_nz * 3.664, (5, 50, 95)))
pl.legend()

# p5 and median are similar to flat10 and 1pctCO2 for TCRE_up, but here we have a bigger estimate for p95

In [ ]:
# ratio
pl.hist(sl_down_nz / tcre_up_at_net_zero_scen_means, bins=np.arange(0.5, 1.45, 0.05))
np.percentile(sl_down_nz / tcre_up_at_net_zero_scen_means, (5, 50, 95))

In [ ]:
# check wrt flat 10
flat10_df = pd.read_csv('../data/flat10_key-metrics_fair2.1.3_cal1.4.1.csv')

In [ ]:
pl.scatter(flat10_df.zec50, ic_down_nz)
#pl.scatter(flat10_df.zec100, ic_down_nz)
pl.plot((-0.3, 0.7), (-0.3, 0.7), ls="--", color='k')
pl.xlim(-0.3, 0.7)
pl.ylim(-0.3, 0.7)
pl.xlabel('ZEC$_{50}$, flat10-MIP')
pl.ylabel('ZEC, regression method')
pl.title('Comparison of ZEC estimates from FaIR')
pl.text(0.6, 0.6, '1:1', rotation=45, size=8, backgroundcolor="w", ha='center', va='center')
pl.savefig('../plots/zec-comparison.pdf')
pl.savefig('../plots/zec-comparison.png')

In [ ]:
tcre_down_flat10 = (flat10_df.tr0 - (flat10_df.tcre+flat10_df.tr1000))/-1
pl.scatter(tcre_down_flat10, sl_down_nz*3.664)
pl.plot((0.75, 3), (0.75, 3), ls="--", color='k')
pl.xlim(0.75, 3)
pl.ylim(0.75, 3)
pl.xlabel('TCRE_down, flat10')
pl.ylabel('TCRE_down, regression method')

In [ ]:
pl.hist(tcre_down_flat10/flat10_df.tcre, bins=np.arange(0.5, 1.4, 0.05), alpha=0.5, label='flat10')
pl.hist(sl_down_nz/tcre_up_at_net_zero_scen_means, bins=np.arange(0.5, 1.4, 0.05), alpha=0.5, label='regression')
print(np.percentile(tcre_down_flat10/flat10_df.tcre, (5, 50, 95)))
print(np.percentile(sl_down_nz/tcre_up_at_net_zero_scen_means, (5, 50, 95)))
pl.legend()